In [3]:
from os import getcwd
import numpy as np
import pylab as plt
from math import log10
from emcee import EnsembleSampler

from scipy.interpolate import interp1d as interp
from scipy.optimize import brentq, minimize_scalar

from profiles import build_profile, build_kernel
from dispersion import SphericalJeansDispersion
from likelihood import GaussianLikelihood

%matplotlib inline

In [ ]:
rh = 0.04
D = 39.81
theta = 2*rh/D

dm = build_profile('NFW')           # NFW profile for DM
st = build_profile('plummer',rh=rh) # Plummer Stellar profile
kr = build_kernel('iso')            # isotropic kernel
dwarf_props = {'D':D, 'theta':theta, 'rt':np.inf, 'with_errs':False}
Sigma = SphericalJeansDispersion(dm, st, kr, dwarf_props)

directory = '/home/andrea/Desktop/work/DWARF/'
R, v = np.loadtxt(directory+'dsphsim/Ret2_data/dsph_001.txt',usecols=(5, 7),unpack=True)
vnan = np.isnan(v)
v = v[:10]#[~vnan]
R = R[:10]#[~vnan]
dv = np.zeros_like(v)

LL = GaussianLikelihood([R, v, dv, 0.], Sigma)
LL.set_free('dm_a')
LL.set_free('dm_b')
LL.set_free('dm_c')
LL.set_free('dm_r0')

In [ ]:
def lnprior(theta, freeJ=False):
    if freeJ:
        J, r, a, b, c = theta
        if -3. < r < 2. and 0.5 < a < 3. and 2. < b < 6. and .0 < c < 1.2 and 12 < J < 22:
            return 0.0
        return -np.inf
    r, a, b, c = theta
    if -3. < r < 2. and 0.5 < a < 3. and 2. < b < 6. and .0 < c < 1.2:
        return 0.0
    return -np.inf

def loglike(theta, *args, freeJ=False):
    if freeJ:
        J, r, a, b, c = theta
        return LL(a, b, J, 10**r, c)
    J, = args
    r, a, b, c = theta
    return LL(a, b, J, 10**r, c)

def lnprob(theta, *args, freeJ=False):
    if freeJ:
        lp = lnprior(theta)
        if not np.isfinite(lp):
            return -np.inf
        return lp + loglike(theta, J)
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + loglike(theta, J)

In [ ]:
ndim = len(LL.free_pars)-1
nwalkers = ndim*4

p0 = [-1., 1., 3., 1.]
pos = [ p0 + 0.1*np.random.randn(ndim) for i in range(nwalkers)]

J = 16.72

# Initialize the MPI-based pool used for parallelization.
pool = MPIPool()

if not pool.is_master():
    # Wait for instructions from the master process.
    pool.wait()
    sys.exit(0)

sampler = EnsembleSampler(nwalkers, ndim, lnprob, args=(J,), pool=pool)

# Run 100 steps as a burn-in.
pos, prob, state = sampler.run_mcmc(pos, 100)

# Reset the chain to remove the burn-in samples.
sampler.reset()

# Starting from the final position in the burn-in chain, 
# sample for 1000 steps.
sampler.run_mcmc(pos, 1000, rstate0=state)

np.save('samples', sampler.chain)
np.save('lnprobs', sampler.lnprobability)